<a href="https://colab.research.google.com/github/Ayanlola2002/Data-Science-Nigeria/blob/master/DSN_BOOTCAMP_2020_Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp= IterativeImputer()
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMModel,LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
!pip install vecstack
from vecstack import stacking
#pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [4]:
train_df=pd.read_csv('/content/drive/My Drive/DSN_BOOTCAMP/Train.csv')
test_df=pd.read_csv('/content/drive/My Drive/DSN_BOOTCAMP/Test.csv')

In [5]:

train_df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [6]:
test_df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [7]:
Application_id=test_df['Applicant_ID']

In [8]:
 #train_df.dropna(thresh=train_df.shape[1]*0.7,how='all',axis=0,inplace=True)

In [9]:

print(train_df.shape)
print(test_df.shape)

(56000, 52)
(24000, 51)


In [10]:
train_df.shape

(56000, 52)

In [11]:
train_df.default_status.unique()

array(['no', 'yes'], dtype=object)

In [12]:
#target varriable
train_df.loc[train_df.default_status=='no','default_status']=0
#target varriable
train_df.loc[train_df.default_status=='yes','default_status']=1

In [13]:
#saving the order Id column and predictor column
#Application_id=test_df['Applicant_ID']
y_train=train_df.iloc[:,-1].astype('int')

In [14]:
y_train

0        0
1        0
2        1
3        0
4        0
        ..
55995    0
55996    1
55997    0
55998    0
55999    0
Name: default_status, Length: 56000, dtype: int64

In [15]:
#type(train_df.isnull().sum())
train_df.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [17]:
train_df.form_field40.unique()
#fill form_field40 with mean,median

array([     nan, 149.6805, 150.0135, ..., 110.334 , 178.371 , 158.025 ])

In [18]:
train_df.form_field47.unique()

array(['charge', 'lending'], dtype=object)

In [19]:
train_df.form_field31.unique()
#mean,median
#its also better to check the distribution with respect to some other fields.
#bivariate analysis

array([   nan, 14508.,  5202., 13413.,   273., 17430., 21990., 15879.,
       11499.,  8031., 31845., 17703., 12684., 13506.,  8943.,  4014.,
       14874.,  1917.,  9945.,  8121.,  5019.,  6024., 11955., 17064.,
       10767., 12867.,  5748.,  6114., 16152., 10584.,  6936.,  3834.,
       22629.,  6204., 10494., 15057., 15969., 21354., 20622., 18069.,
        3468.,  8670., 12135., 23451.,  4746., 10311., 16974.,  7299.,
       16791., 13869., 10404., 10677.,  5385., 10128.,  9489., 15786.,
       11316.,   822., 11772., 13140.,  8577., 12318.,  3375.,  7026.,
        6570.,  2829.,  7755.,  9399., 22083.,  1368., 10038.,   456.,
        2556., 14145.,  9765., 15696., 13779., 15603.,  1185., 15513.,
       20805.,  4929.,  2463., 14418.,  7119.,  6297., 19164., 19983.,
        8304.,  3741., 20349.,  6753.,  4653., 11679., 12501.,   729.,
       18798., 10221., 29019., 11406., 27465.,  7575., 25368., 21810.,
       14235., 17154.,  9672.,  4470., 21444., 23724., 11589.,  5109.,
      

In [20]:
(train_df.form_field6.isnull().sum()/len(train_df))*100

23.857142857142858

In [21]:
#filling field6 and field7 nan values with zeros
train_df.fillna({'form_field6':0,'form_field7':0},inplace=True)
test_df.fillna({'form_field6':0,'form_field7':0},inplace=True)

In [22]:
#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train_df.drop("Applicant_ID", axis = 1, inplace = True)
test_df.drop("Applicant_ID", axis = 1, inplace = True)

In [23]:
# let's delete the last column from the dataset to  concat train and test
train_df = train_df.drop(['default_status'], axis = 1)

In [24]:
train_df.shape


(56000, 50)

In [25]:
# lets concat the train and test sets for preprocessing and visualizations
all_data = pd.concat([train_df, test_df], axis = 0)

# let's check the shape
all_data.shape

(80000, 50)

In [26]:
categorical_columns = []
numeric_columns = []
for c in all_data.columns:
    if all_data[c].map(type).eq(str).any(): #check if there are any strings in column
        categorical_columns.append(c)
    else:
        numeric_columns.append(c)

#create two DataFrames, one for each data type
data_numeric = all_data[numeric_columns]
data_categorical = pd.DataFrame(all_data[categorical_columns])




#you could do something like one-hot-encoding of data_categorical here

#join the two masked dataframes back together
#data_joined = pd.concat([data_numeric, data_categorical], axis = 1)

In [27]:
#imputing NaN values with mean/median
imp =SimpleImputer(missing_values=np.nan, strategy='median')
data_numeric = pd.DataFrame(imp.fit_transform(data_numeric), columns = data_numeric.columns) #only apply imputer to numeric columns
data_numeric.reset_index(drop=True, inplace=True)
data_categorical.reset_index(drop=True, inplace=True)

In [28]:
print(data_categorical.shape)
print(data_numeric.shape)

(80000, 1)
(80000, 49)


In [29]:
#join the two masked dataframes back together
data_joined = pd.concat([data_numeric, data_categorical], axis = 1)

In [30]:
data_joined.shape

(80000, 50)

In [31]:

data_joined

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50,form_field47
0,3436.0,0.28505,1.6560,0.0000,0.0000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139.0,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,5961.6375,5550.000,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,10584.0,1052.0,8.0000,2.0,0.0,12.0,12.0,4.0,0.0,150.0000,118.32525,0.392854,2.02,0.711632,0.0,0.0,316.707639,1.129518,0.044335,charge
1,3456.0,0.67400,0.2342,0.0000,0.0000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427.0,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.2400,14584.290,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,10584.0,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,150.0000,118.32525,0.314281,8.08,0.183584,0.0,0.0,349.805730,1.620483,0.322436,charge
2,3276.0,0.53845,3.1510,0.0000,6.2820,0.0,956940.0,963942.0,192944.0,1079864.0,692420.0,1158247.5,0.0,16576457.0,22452691.0,0.0,0.0,0.0,0.0,0.0,89.535,5961.6375,5550.000,60.660,1597.5,4746.0,3164.0,60.0,15696.0,5658.0,14508.0,70.0,0.6666,0.0,0.0,12.0,0.0,6.0,0.0,149.6805,118.32525,0.162965,18.18,0.791136,0.0,0.0,316.707639,1.513370,0.011640,charge
3,3372.0,0.17005,0.5050,0.0000,0.0000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,692420.0,771776.0,1186839.0,48187375.0,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.0000,0.000,85.800,1701.0,4014.0,2676.0,60.0,11679.0,5658.0,10584.0,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,150.0000,118.32525,0.488884,2.02,0.685168,0.0,0.0,89.940100,0.664452,0.082729,charge
4,3370.0,0.77270,1.1010,0.0000,0.0000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427.0,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.1900,10326.450,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,118.32525,0.275000,12.12,0.438168,0.0,0.0,97.887502,1.427891,0.045630,charge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,3258.0,0.37925,0.5698,5.1282,0.0000,0.0,2280318.0,963942.0,5512519.0,2280318.0,1743887.0,1158247.5,0.0,21973443.0,126128971.0,0.0,0.0,0.0,0.0,0.0,89.535,5961.6375,5550.000,60.660,547.5,8214.0,5110.0,304.0,3741.0,2068.0,10584.0,24.0,0.3334,0.0,0.0,6.0,0.0,6.0,0.0,150.0000,92.69100,1.100000,0.00,0.800000,0.0,1.0,316.707639,0.000000,0.169760,charge
79996,3514.0,0.95260,0.0000,0.0000,0.0000,1494538.0,4773419.0,4773419.0,89718571.0,12999991.0,692420.0,48989882.0,97696961.0,96375139.0,84083128.0,0.0,0.0,0.0,3.0,3.0,123.195,11663.0550,5550.000,123.195,1485.0,6387.0,4258.0,60772.0,14691.0,3286.0,10584.0,586.0,14.1666,0.0,0.0,10.0,2.0,8.0,0.0,150.0000,125.02200,0.000000,12.12,0.196400,0.0,0.0,4200.072959,3.000000,0.930931,charge
79997,3302.0,0.05755,4.2394,0.0000,25.4376,0.0,49403.0,963942.0,5512519.0,49403.0,215895.0,1158247.5,0.0,21973443.0,84083128.0,0.0,0.0,0.0,0.0,1.0,89.535,13080.0000,5550.000,60.660,2304.0,8214.0,5110.0,60.0,10038.0,5658.0,5109.0,202.0,5.3334,0.0,0.0,8.0,4.0,2.0,0.0,150.0000,118.32525,1.100000,1.01,0.800000,0.0,0.0,316.707639,4.855060,0.169760,charge
79998,3334.0,0.75670,1.4428,0.0000,0.0000,0.0,2513329.0,37733.0,393668.0,2551062.0,121757.0,355935.0,673748.0,16576457.0,84083128.0,1.0,2.0,2.0,2.0,2.0,129.435,13024.9950,13024.995,138.450,706.5,2463.0,1642.0,0.0,6297.0,2312.0,10584.0,62.0,2.6666,0.0,0.0,12.0,0.0

In [32]:
#label encoding categorical varriables

le = LabelEncoder()
data_joined['form_field47'] = le.fit_transform(data_joined['form_field47'])
# removing unnecassary columns

data_joined =data_joined.drop(['form_field47'], axis = 1)

In [33]:

# separating train and test datasets from data

x_train = data_joined.iloc[:56000,:]
x_test = data_joined.iloc[56000:,:]

# checking the shape of train and test
print("Shape of train :", x_train.shape)
print("Shape of test :", x_test.shape)

Shape of train : (56000, 49)
Shape of test : (24000, 49)


In [34]:
# train and valid sets from train
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)

# checking the shapes
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(44800, 49)
(44800,)
(11200, 49)
(11200,)


In [35]:

# standardization techniques

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(x_test)

In [36]:
import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

In [37]:
'''# k-Nearest Neighbors algorithm

knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid={'n_neighbors': [2,3,4,5]}, cv=10).fit(x_train,y_train)
acc_knn = round(knn.score(x_train,y_train) * 100, 2)
print(acc_knn, knn.best_params_)

acc_test_knn = round(knn.score(x_valid,y_valid) * 100, 2)
print(acc_test_knn)'''

"# k-Nearest Neighbors algorithm\n\nknn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid={'n_neighbors': [2,3,4,5]}, cv=10).fit(x_train,y_train)\nacc_knn = round(knn.score(x_train,y_train) * 100, 2)\nprint(acc_knn, knn.best_params_)\n\nacc_test_knn = round(knn.score(x_valid,y_valid) * 100, 2)\nprint(acc_test_knn)"

In [38]:
# Logistic Regression
'''logreg = LogisticRegression()
logreg.fit(x_train,y_train)
acc_log = round(logreg.score(x_train,y_train) * 100, 2)
print(acc_log)

acc_test_log = round(logreg.score(x_valid,y_valid) * 100, 2)
print(acc_test_log)'''

'logreg = LogisticRegression()\nlogreg.fit(x_train,y_train)\nacc_log = round(logreg.score(x_train,y_train) * 100, 2)\nprint(acc_log)\n\nacc_test_log = round(logreg.score(x_valid,y_valid) * 100, 2)\nprint(acc_test_log)'

In [39]:
#Adaboost tuning
'''def hyperopt_ab_score(params):
    clf = AdaBoostClassifier(**params)
    current_score = cross_val_score(clf, x_train,y_train, cv=10).mean()
    print(current_score, params)
    return current_score 
 
space_ab = {
            'n_estimators': hp.choice('n_estimators', range(50, 1000,50)),
            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001)       
        }
 
best = fmin(fn=hyperopt_ab_score, space=space_ab, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)'''

"def hyperopt_ab_score(params):\n    clf = AdaBoostClassifier(**params)\n    current_score = cross_val_score(clf, x_train,y_train, cv=10).mean()\n    print(current_score, params)\n    return current_score \n \nspace_ab = {\n            'n_estimators': hp.choice('n_estimators', range(50, 1000,50)),\n            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001)       \n        }\n \nbest = fmin(fn=hyperopt_ab_score, space=space_ab, algo=tpe.suggest, max_evals=10)\nprint('best:')\nprint(best)"

In [40]:
'''params = space_eval(space_ab, best)
params'''

'params = space_eval(space_ab, best)\nparams'

In [41]:
# AdaBoost Classifier

'''Ada_Boost = AdaBoostClassifier(**params)
Ada_Boost.fit(x_train,y_train)
acc_AdaBoost = round(Ada_Boost.score(x_train,y_train) * 100, 2)
print(acc_AdaBoost)

acc_test_AdaBoost = round(Ada_Boost.score(x_valid,y_valid) * 100, 2)
print(acc_test_AdaBoost)'''

'Ada_Boost = AdaBoostClassifier(**params)\nAda_Boost.fit(x_train,y_train)\nacc_AdaBoost = round(Ada_Boost.score(x_train,y_train) * 100, 2)\nprint(acc_AdaBoost)\n\nacc_test_AdaBoost = round(Ada_Boost.score(x_valid,y_valid) * 100, 2)\nprint(acc_test_AdaBoost)'

In [ ]:
#GradientBoost Tuning
def hyperopt_gb_score(params):
    clf = GradientBoostingClassifier(**params)
    current_score = cross_val_score(clf,x_train,y_train, cv=10).mean()
    print(current_score, params)
    return current_score 
 
space_gb = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
            'max_depth': hp.choice('max_depth', np.arange(2, 10, dtype=int))            
        }
 
best = fmin(fn=hyperopt_gb_score, space=space_gb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

In [ ]:
params = space_eval(space_gb, best)
params

In [ ]:
# Gradient Boosting Classifier

gradient_boosting = GradientBoostingClassifier(**params)
gradient_boosting.fit(x_train,y_train)
acc_gradient_boosting = round(gradient_boosting.score(x_train,y_train) * 100, 2)
print(acc_gradient_boosting)

acc_test_gradient_boosting = round(gradient_boosting.score(x_valid,y_valid) * 100, 2)
print(acc_test_gradient_boosting)

In [ ]:
#Extratrees tuning
    def hyperopt_etc_score(params):
    clf = ExtraTreesClassifier(**params)
    current_score = cross_val_score(clf, x_train,y_train, cv=10).mean()
    print(current_score, params)
    return current_score 
 
space_etc = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
            'max_features': hp.choice('max_features', np.arange(2, 17, dtype=int)),
            'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(1, 5, dtype=int)),
            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),
            'max_features': None # for small number of features
        }
 
best = fmin(fn=hyperopt_etc_score, space=space_etc, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

In [ ]:
params = space_eval(space_etc, best)
params

In [ ]:
# Extra Trees Classifier

extra_trees_classifier = ExtraTreesClassifier(**params)
extra_trees_classifier.fit(x_train,y_train)
acc_etc = round(extra_trees_classifier.score(x_train,y_train) * 100, 2)
print(acc_etc)

acc_test_etc = round(extra_trees_classifier.score(x_valid,y_valid) * 100, 2)
print(acc_test_etc)

In [ ]:
   #LGBoost Tuning
   def hyperopt_lgb_score(params):
    clf = LGBMClassifier(**params)
    current_score = cross_val_score(clf,x_train,y_train, cv=10).mean()
    print(current_score, params)
    return current_score 
 
space_lgb = {
            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),
            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),
            'num_leaves': hp.choice('num_leaves', 2*np.arange(2, 2**11, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),
            'objective': 'binary',
            'boosting_type': 'gbdt',
            }
 
best = fmin(fn=hyperopt_lgb_score, space=space_lgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

In [ ]:
params = space_eval(space_lgb, best)
params

In [ ]:
LGB_Classifier = LGBMClassifier(**params)
LGB_Classifier.fit(x_train,y_train)
acc_LGB_Classifier = round(LGB_Classifier.score(x_train,y_train) * 100, 2)
print(acc_LGB_Classifier)

In [ ]:
acc_test_LGB_Classifier = round(LGB_Classifier.score(x_valid,y_valid) * 100, 2)
print(acc_test_LGB_Classifier)

In [ ]:
def hyperopt_xgb_score(params):
    clf = XGBClassifier(**params)
    current_score = cross_val_score(clf, x_train,y_train, cv=10).mean()
    print(current_score, params)
    return current_score 
 
space_xgb = {
            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),
            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.005),
            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.005),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.005),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1,
            'missing': None
        }
 
best = fmin(fn=hyperopt_xgb_score, space=space_xgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

In [ ]:
params = space_eval(space_xgb, best)
params

In [ ]:
XGB_Classifier = XGBClassifier(**params)
XGB_Classifier.fit(x_train,y_train)
acc_XGB_Classifier = round(XGB_Classifier.score(x_train,y_train) * 100, 2)
print(acc_XGB_Classifier)

In [ ]:
acc_test_XGB_Classifier = round(XGB_Classifier.score(x_valid,y_valid) * 100, 2)
print(acc_test_XGB_Classifier)

In [ ]:
# Random Forest

random_forest = GridSearchCV(estimator=RandomForestClassifier(), param_grid={'n_estimators': [100, 300,400,500]}, cv=5).fit(x_train,y_train)
random_forest.fit(x_train,y_train)
acc_random_forest = round(random_forest.score(x_train,y_train) * 100, 2)
print(acc_random_forest,random_forest.best_params_)

In [ ]:
acc_test_random_forest = round(random_forest.score(x_valid,y_valid) * 100, 2)
print(acc_test_random_forest)

In [ ]:
#models
models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    
    ExtraTreesClassifier(n_estimators=700,
                                     max_features=49,
                                     criterion= 'entropy',
                                     min_samples_split=2,
                                     max_depth=49,
                                     min_samples_leaf=2, ## the default is 1 (note we use more than one sample for the split)
                                     class_weight='balanced_subsample',
                                     random_state=1,
                                     verbose=1),

        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
     XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=3),
          
    LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
]

In [ ]:
S_train, S_test = stacking(models,                   
                           x_train,y_train,x_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=True,
                      
         
                           save_dir=None, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [10]

model  0:     [KNeighborsClassifier]
    fold  0:  [1.78716323]
    fold  1:  [1.86537231]
    fold  2:  [1.88417833]
    fold  3:  [1.85253091]
    ----
    MEAN:     [1.84731119] + [0.03650484]
    FULL:     [1.84731119]

model  1:     [DecisionTreeClassifier]
    fold  0:  [9.84358747]
    fold  1:  [9.84975511]
    fold  2:  [9.83442359]
    fold  3:  [9.91756294]
    ----
    MEAN:     [9.86133228] + [0.03291988]
    FULL:     [9.86133228]

model  2:     [LogisticRegression]
    fold  0:  [0.42787816]
    fold  1:  [0.43633208]
    fold  2:  [0.43203154]
    fold  3:  [0.43182597]
    ----
    MEAN:     [0.43201694] + [0.00299110]
    FULL:     [0.43201694]

model  3:     [LinearDiscriminantAnalysis]
    fold  0:  [0.43329501]
    fold  1:  [0.44346331]
    fold  2:  [0.43839248]
    fold  3:  [0.44082236]
    ----
    MEAN:     [0.43899329] + [0.00374693]
   

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    5.3s finished


    fold  0:  [0.43038149]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    5.2s finished


    fold  1:  [0.42925172]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:  2.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    5.3s finished


    fold  2:  [0.43411741]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:  2.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:    5.3s finished


    fold  3:  [0.43113358]
    ----
    MEAN:     [0.43122105] + [0.00180137]
    FULL:     [0.43122105]

model  7:     [RandomForestClassifier]
    fold  0:  [0.45110488]
    fold  1:  [0.45659969]
    fold  2:  [0.45289873]
    fold  3:  [0.45297943]
    ----
    MEAN:     [0.45339568] + [0.00199585]
    FULL:     [0.45339568]

model  8:     [XGBClassifier]
    fold  0:  [0.42149575]
    fold  1:  [0.42816280]
    fold  2:  [0.42528366]
    fold  3:  [0.42406260]
    ----
    MEAN:     [0.42475120] + [0.00239764]
    FULL:     [0.42475120]

model  9:     [LGBMClassifier]
    fold  0:  [0.42076110]
    fold  1:  [0.42500264]
    fold  2:  [0.42466158]
    fold  3:  [0.42338329]
    ----
    MEAN:     [0.42345215] + [0.00166683]
    FULL:     [0.42345215]



In [ ]:
model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=4,
 min_child_weight=7.0,
 gamma=0.3,
 subsample=0.9,
 colsample_bytree=0.9,
 reg_alpha=0.03,
 objective= 'binary:logistic',
 seed=20
 )
    
model = model.fit(S_train, y_train)
y_pred = model.predict_proba(S_test)[:,1]
#print('Final prediction score: [%.8f]' % accuracy_score(y_valid, y_pred))

In [ ]:

#Prepare our Submission file
my_submission = pd.DataFrame({'Application ID':Application_id, 'default_status':y_pred})
my_submission.to_csv('submission_ayanlola_simple_mean_morestacking4.csv', index=False)